# ICRA Tutorial 4, Contradictory Prior Data

**API Version: JuliaLang NavAbilitySDK.jl**

## Introduction

Many industrial robotic systems today are constrained to very rigid automation regimes.  Production/manufacturing lines are often difficult to reconfigure to different or new operations.  It can also be challenging for the robotic systems to react in real-time based on sensor data.  One of the critical requirements for expanding robotic operations beyond the rigid brute force automation is the ability to incorporate/restrain robotic operations with prior knowledge or data.  This tutorial shows how prior data can be used to support as well as constrain robotic state-estimation tasks, while also gaining robustness to contradictory data.  This tutorial will show how to combine prior and new measurement data through multi-hypothesis robustness.

## Application: Beyond "Brute-Force" Automation while Retaining Prior Knowledge

Conventional manufacturing and warehouse automation technologies operate via "brute force" or "playback" control strategies where robot operations follow a very narrow preprogrammed path.  One of the major challenges in AI development for manufacturing and warehouse applications is to make the high value robotic equipment more flexible / reconfigurable so that the systems can more rapidly be reconfigured between different tasks.  

One of the key challenges to expanding the automation software capabilities requires a balance between 
- programming the robot with human readible prior information about the task; as well as 
- letting the robotic automation handle more of the fine grain numerical operations relating to the task at hand.

The figure below shows a motivational example where prior knowledge---i.e. the corners of a rectangle and door opening---is to be used for robotic mapping operations.

![Tutorial 4 Room Prior](../../static/icra-4/room-prior.png)

The figure above illustrates some robotic equipment (a.k.a. the robot) which must be localized relative to a known rectangular shape.  For simplicity, we introduce the door orientation as the only ambiguity which must be mapped by the robot -- i.e. we know there is a door, but have not yet verified the hinging orientation.

The challenge of this tutorial is localizing and mapping the robot relative to the object as efficiently as possible, and have the solution be robust to possibly contradictory prior data.  For the sake of illustration, we simplify the possible contradiction to the door orientation only, and invite the interested reader to see the [NavAbility Construction Application example](https://www.navability.io/applications/construction/) and associated publications for a more elaborate discussion on navigation-affordances.

### Prior (Contradictory) Data

This tutorial will demonstrate through a simplified example how prior (contradictory) data can be leveraged in a localization and mapping solution using factor graphs.  We want to pre-load our robotic system with some basic information about the environment in which it will be operating so that it can more easily resolve it's position and navigate around.

#### Room with a Door

Prior info:
- A rectangular room with the 4 corners as defining known navigation affordances,
- An orientation sensitive door affordance which should* be hinged on the left side (which is buried in the data as an easter egg for the solver to discover as valid or contradictory).

## Import Packages

The first step load all the necessary packages, as listed in code blocks hereafter.  The following commented code block is only necessary if your environment does not have the necessary packages installed for whatever reason (uncomment and run once if package are not installed).

In [ ]:
# # install if necessary
# import Pkg; Pkg.add("NavAbilitySDK")

In [1]:
using NavAbilitySDK

## Build a Multi-hypothesis Factor Graph

We start with an empty factor graph object, which is created with default solver parameters.

In [5]:
# you need a unique userId:robotId, and can keep using that across all tutorials
userId = "guest@navability.io"
robotId = "SDKjl_"*(string(uuid4())[1:4])

# also create a client connection
client = NavAbilityHttpsClient()

# You'll need a unique session number each time you run a new graph
sessionId = "Tutorial4_"*(string(uuid4())[1:4])

# context is the object to use below
context = Client(userId,robotId,sessionId)

Client: User=guest@navability.io, Robot=SDKjl_13e4, Session=Tutorial4_87e5

Also a few basic noise parameters

In [6]:
prior_distr= diagm([0.1, 0.1, 0.01].^2)
dual_distr= diagm([1, 1, sqrt(pi)].^2)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  3.14159

### Loading Known Prior Data

Next, add to the factor graph the four corner prior knowledge about the room, namely `c0`, `c1`, `c2`, `c3`, each with their expected location in the world via `PriorPose2` unary factor:

In [7]:
resultIds = Task[]

addVariable(client, context, Variable("c0", :Pose2))
f = Factor("c0f1", "PriorPose2", ["c0"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([0.,0, 0], prior_distr)))
push!(resultIds, addFactor(client, context, f))

addVariable(client, context, Variable("c1", :Pose2))
f = Factor("c1f1", "PriorPose2", ["c1"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([20.,0, pi/2], prior_distr)))
push!(resultIds, addFactor(client, context, f))

addVariable(client, context, Variable("c2", :Pose2))
f = Factor("c2f1", "PriorPose2", ["c2"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([20.,10, pi], prior_distr)))
push!(resultIds, addFactor(client, context, f))

addVariable(client, context, Variable("c3", :Pose2))
f = Factor("c3f1", "PriorPose2", ["c3"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([0.,10, -pi/2], prior_distr)))
push!(resultIds, addFactor(client, context, f))

4-element Vector{Task}:
 Task (runnable) @0x00007f94e2c87de0
 Task (runnable) @0x00007f9472feeca0
 Task (runnable) @0x00007f947304aca0
 Task (runnable) @0x00007f94730b2ca0

Also add the prior information regarding the door, and note for this simplified tutorial is using two affordance variables.  Two variables is part of the _affordance-duality_ construct where contradictions between prior known and ultimately discovered data can manifest in either the prior or dual variables, respectfully:

In [8]:
addVariable(client, context, Variable("door_prior", :Pose2))
f = Factor("door_priorf1", "PriorPose2", ["door_prior"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([20., 4, 0], prior_distr)))
push!(resultIds, addFactor(client, context, f))

addVariable(client, context, Variable("door_dual", :Pose2))
f = Factor("door_dualf1", "PriorPose2", ["door_dual"], PriorPose2Data(Z=NavAbilitySDK.FullNormal([20., 4, pi], dual_distr)))
push!(resultIds, addFactor(client, context, f))

6-element Vector{Task}:
 Task (done) @0x00007f94e2c87de0
 Task (done) @0x00007f9472feeca0
 Task (done) @0x00007f947304aca0
 Task (done) @0x00007f94730b2ca0
 Task (runnable) @0x00007f9473187990
 Task (runnable) @0x00007f94731d3990

### Solving the Room-only Graph

Let's make sure our prior data makes sense by solving the factor graph with the available data:

In [9]:
# wait for graph modifications to complete
waitForCompletion(client, resultIds; expectedStatuses=["Complete"], maxSeconds=180)

println("starting solve")
push!(resultIds,
  solveSession(client, context)
);

# Give it a few seconds JIT compiling during first run.
println("running solve...")

starting solve
running solve...


While the solve is running, let's look so long at the factor graph so far, and look at the map thereafter...

#### The Room-only Graph Structure

The basic room structure in the factor graph is also of interest.  Let's go ahead and look at what the graph structure looks like:

In [ ]:
# Click on the generated factor graph graphic to open NavAbility App visualization
GraphVizApp(context)

The graph plot above shows the simple factor graph so far, where four corner and dual door variables each have a prior factor.  We are now ready to introduce new robotic measurements into the factor graph.

#### Map of Prior Data

After completing the earlier solve, we can use the NavAbility WebApp to visualize the numerical values of the variables in the graph.

In [11]:
# Click on the generated factor graph graphic to open NavAbility App visualization
MapVizApp(context)

https://app.navability.io/cloud/map/?userId=guest@navability.io&robotStartsWith=SDKjl_13e4&sessionStartsWith=Tutorial4_87e5


[![Navigate to Factor Graph](http://www.navability.io/wp-content/uploads/2022/03/geometric_map.png)](https://app.navability.io/cloud/map/?userId=guest@navability.io&robotStartsWith=SDKjl_13e4&sessionStartsWith=Tutorial4_87e5)

The plot shows the four corners as a rectangle between `(0,0)` and `(20,10)`, as well as a door at `(20,4)`.  We also confirm the door prior data orientation which to the best of our current knowledge is **left hinged**:

In [10]:
# Click on the generated factor graph graphic to open NavAbility App visualization
GraphVizApp(context)

https://app.navability.io/cloud/graph/?userId=guest@navability.io&robotStartsWith=SDKjl_13e4&sessionStartsWith=Tutorial4_87e5


[![Navigate to Factor Graph](http://www.navability.io/wp-content/uploads/2022/03/factor_graph.png)](https://app.navability.io/cloud/graph/?userId=guest@navability.io&robotStartsWith=SDKjl_13e4&sessionStartsWith=Tutorial4_87e5)

The graph plot above shows the simple factor graph so far, where four corner and dual door variables each have a prior factor.  We are now ready to introduce new robotic measurements into the factor graph.

## Add Robotic Measurements (`multihypo=`)

We would like to localize our robotic system relative to the available prior data above.  The first thing the robot does is measure one of the corners relative to it's own position (i.e. **a relative, not global, measurement**).  Since we do not know which of the corners this measurement relates to, we will use the `multihypo=` keyword with equal `1/4` probability to each of the four known corners:

In [13]:
# add the first robot pose as x0
addVariable(client, context, Variable("x0", :Pose2))

f = Factor(
  "x0c0c1c2c3f1", 
  "Pose2Pose2", 
  ["x0", "c0", "c1", "c2", "c3"], 
  Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([-2.,-2, 0], diagm([0.5, 0.5, 0.05].^2))),
  multihypo=[1.0, 0.25, 0.25, 0.25, 0.25]
)
push!(resultIds, addFactor(client, context, f))

8-element Vector{Task}:
 Task (done) @0x00007f94e2c87de0
 Task (done) @0x00007f9472feeca0
 Task (done) @0x00007f947304aca0
 Task (done) @0x00007f94730b2ca0
 Task (done) @0x00007f9473187990
 Task (done) @0x00007f94731d3990
 Task (done) @0x00007f9473049e40
 Task (runnable) @0x00007f94731fcd00

Above we first added a new robot 'pose' variable `x0`.  We then added a relative `Pose2Pose2` factor which represents a rigid transform between the robot and the measured corner.  Note, the measurement is local to the robot (i.e. not a global metrology-type measurement).  Also note that the `Pose2Pose2` factor is usually binary between two variables but now, with the `multihypo=` keyword, is associated with five variables.  The first is the robot pose `x0` with `100%` certainty, while the next four corner affordance variables have fractional association certainty of `25%` each.

#### Solve and Visualize the Multi-hypo Factor Graph 

Let's start a new solve and look at the factor graph structure while the computation is running: 

In [14]:
# wait for graph modifications to complete
waitForCompletion(client, resultIds; expectedStatuses=["Complete"], maxSeconds=180)

push!(resultIds,
  solveSession(client, context)
)
println("running solve...")

running solve...


They multi-hypo factor is represented in the factor graph as any other n-ary factor

In [12]:
# Click on the generated factor graph graphic to open NavAbility App visualization
GraphVizApp(context)

https://app.navability.io/cloud/graph/?userId=guest@navability.io&robotStartsWith=SDKjl_13e4&sessionStartsWith=Tutorial4_87e5


[![Navigate to Factor Graph](http://www.navability.io/wp-content/uploads/2022/03/factor_graph.png)](https://app.navability.io/cloud/graph/?userId=guest@navability.io&robotStartsWith=SDKjl_13e4&sessionStartsWith=Tutorial4_87e5)

Let's recap what the factor represents and what we are expecting to see.  We know there are four corners to a room and we have one relative measurement to a corner.  Therefore, the robot should be relatively positioned to one of the four known corners -- i.e. we expect to see a multi-modal posterior belief estimate on the robot pose `x0`.  Let's plot the numerical results to see:

In [15]:
# Click on the generated factor graph graphic to open NavAbility App visualization
MapVizApp(context)

https://app.navability.io/cloud/map/?userId=guest@navability.io&robotStartsWith=SDKjl_13e4&sessionStartsWith=Tutorial4_87e5


[![Navigate to Factor Graph](http://www.navability.io/wp-content/uploads/2022/03/geometric_map.png)](https://app.navability.io/cloud/map/?userId=guest@navability.io&robotStartsWith=SDKjl_13e4&sessionStartsWith=Tutorial4_87e5)

As predicted, there are four separate modes in the posterior marginal estimate of `x0`, one nearby each of the four known corners of the room.  Note that each of these modes also have a marginal belief over the orientation of the robot which corresponds to the relative transform to the measured corner.

The critcal point here is that even though the factor graph does not have a unimodal solution, we are able to perform a solve on the graph and recover the correct non-Gaussian marginal posterior result.

## Robot Moves to Position `x1`

The next step towards mapping while localizating the robot relative to prior data is moving the robot and making more measurements.  To get to the second position, we move the robot forward by `2` units and rotate clockwise (around positive `z`) by `90deg`; this transform is captured in another rigid transform `Pose2Pose2` factor to a new variable `x1`.  Once at `x1` the robot makes a second corner measurement for which we again don't know the association:

In [16]:
resultIds = Task[]

# add next pose variable
push!( resultIds, addVariable(client, context, Variable("x1", :Pose2)) )

f = Factor(
  "x0x1f1", 
  "Pose2Pose2", 
  ["x0", "x1"], 
  Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([4.,0, pi/2], diagm([0.5, 0.5, 0.05].^2)))
)
push!( resultIds, addFactor(client, context, f) )

f = Factor(
  "x1c0c1c2c3f1", 
  "Pose2Pose2", 
  ["x1", "c0", "c1", "c2", "c3"], 
  Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([-2.,-4, 0], diagm([0.5, 0.5, 0.05].^2))),
  multihypo=[1.0, 0.25, 0.25, 0.25, 0.25],
  nullhypo=0.5
)
push!( resultIds, addFactor(client, context, f) )

MethodError: MethodError: no method matching Factor(::String, ::String, ::Vector{String}, ::FactorData; multihypo=[1.0, 0.25, 0.25, 0.25, 0.25], nullhypo=0.5)
Closest candidates are:
  Factor(::String, ::String, ::Vector{String}, ::FactorData; tags, timestamp, multihypo) at ~/.julia/packages/NavAbilitySDK/50X4u/src/navability/entities/Factor.jl:186 got unsupported keyword argument "nullhypo"
  Factor(::Any, ::Any, ::Any, ::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) at ~/.julia/packages/NavAbilitySDK/50X4u/src/navability/entities/Factor.jl:24 got unsupported keyword arguments "multihypo", "nullhypo"
  Factor(::String, ::String, !Matched::String, !Matched::Vector{String}, !Matched::FactorData, !Matched::Int64, !Matched::Vector{String}, !Matched::String, !Matched::String) at ~/.julia/packages/NavAbilitySDK/50X4u/src/navability/entities/Factor.jl:24 got unsupported keyword arguments "multihypo", "nullhypo"

Note, that even though we know this second corner measured is a different corner than what we measured first, we lazily added the new corner measurement again as a four way equal `multihypo=` uncertainty.  We help the solver a little bit with this by allowed a bit more freedom in the solution (i.e. an additional source of entropy), but also stating that the interpose measurement might not be wholly correct -- and we do this by including `nullhypo=0.5`.

We are therefore giving the factor graph solver less information than we actually have available.  Let's solve the graph again and see what it finds: 

In [ ]:
solveSession(client, context)


In [ ]:

addVariable(client, context, Variable("x2", :Pose2))

f = Factor("x1x2f1", 
           "Pose2Pose2", 
           ["x1", "x2"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([16.,0, 0], diagm([0.5, 0.5, 0.05].^2))))
addFactor(client, context, f)


f = Factor("x2l0l1l2l3f1", 
           "Pose2Pose2", 
           ["x2", "l0", "l1", "l2", "l3"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([2.,-4, pi/2], diagm([0.5, 0.5, 0.05].^2))),
           multihypo=[1.0, 0.25, 0.25, 0.25, 0.25])
addFactor(client, context, f)





In [ ]:
solveSession(client, context)


In [ ]:


f = Factor("x2l4f1", 
           "Pose2Pose2", 
           ["x2", "l4_h1", "l4_h2"], 
           Pose2Pose2Data(Z=NavAbilitySDK.FullNormal([2.,0, pi], diagm([0.5, 0.5, 0.05].^2))),
           multihypo=[1.0, 0.5, 0.5])
addFactor(client, context, f)


In [ ]:
solveSession(client, context)
